In [2]:
import sys, os
import pandas as pd
import numpy as np

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization,AveragePooling2D
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam
from keras.regularizers import l2
from tensorflow.keras.utils import to_categorical


df=pd.read_csv('C:/Users/prana/OneDrive/Desktop/pro project/fer2013.csv')


X_train,train_y,X_test,test_y=[],[],[],[]

for index, row in df.iterrows():
    val=row['pixels'].split(" ")
    try:
        if 'Training' in row['Usage']:
           X_train.append(np.array(val,'float32'))
           train_y.append(row['emotion'])
        elif 'PublicTest' in row['Usage']:
           X_test.append(np.array(val,'float32'))
           test_y.append(row['emotion'])
    except:
        print(f"error occured at index :{index} and row:{row}")


num_features = 64
num_labels = 7
batch_size = 64
epochs = 50
width, height = 48, 48


X_train = np.array(X_train,'float32')
train_y = np.array(train_y,'float32')
X_test = np.array(X_test,'float32')
test_y = np.array(test_y,'float32')

train_y = to_categorical(train_y, num_classes=num_labels)
test_y = to_categorical(test_y, num_classes=num_labels)


X_train -= np.mean(X_train, axis=0)
X_train /= np.std(X_train, axis=0)

X_test -= np.mean(X_test, axis=0)
X_test /= np.std(X_test, axis=0)

X_train = X_train.reshape(X_train.shape[0], 48, 48, 1)

X_test = X_test.reshape(X_test.shape[0], 48, 48, 1)


model = Sequential()

model.add(Conv2D(64, kernel_size=(3, 3), activation='relu', input_shape=(X_train.shape[1:])))
model.add(Conv2D(64,kernel_size= (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2, 2)))
model.add(Dropout(0.5))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2, 2)))
model.add(Dropout(0.5))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2, 2)))

model.add(Flatten())

#fully connected neural networks
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(num_labels, activation='softmax'))

model.compile(loss=categorical_crossentropy,
              optimizer=Adam(),
              metrics=['accuracy'])

model.fit(X_train, train_y,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(X_test, test_y),
          shuffle=True)

Epoch 1/50


C:\Users\prana\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


449/449 ━━━━━━━━━━━━━━━━━━━━ 51s 110ms/step - accuracy: 0.2557 - loss: 1.7964 - val_accuracy: 0.3940 - val_loss: 1.5704
Epoch 2/50
449/449 ━━━━━━━━━━━━━━━━━━━━ 51s 113ms/step - accuracy: 0.3948 - loss: 1.5347 - val_accuracy: 0.4661 - val_loss: 1.3889
Epoch 3/50
449/449 ━━━━━━━━━━━━━━━━━━━━ 50s 112ms/step - accuracy: 0.4512 - loss: 1.4185 - val_accuracy: 0.4801 - val_loss: 1.3298
Epoch 4/50
449/449 ━━━━━━━━━━━━━━━━━━━━ 51s 113ms/step - accuracy: 0.4758 - loss: 1.3506 - val_accuracy: 0.5035 - val_loss: 1.2769
Epoch 5/50
449/449 ━━━━━━━━━━━━━━━━━━━━ 50s 111ms/step - accuracy: 0.5070 - loss: 1.2794 - val_accuracy: 0.5222 - val_loss: 1.2515
Epoch 6/50
449/449 ━━━━━━━━━━━━━━━━━━━━ 50s 112ms/step - accuracy: 0.5138 - loss: 1.2619 - val_accuracy: 0.5263 - val_loss: 1.2337
Epoch 7/50
449/449 ━━━━━━━━━━━━━━━━━━━━ 50s 111ms/step - accuracy: 0.5346 - loss: 1.2174 - val_accuracy: 0.5405 - val_loss: 1.1994
Epoch 8/50
449/449 ━━━━━━━━━━━━━━━━━━━━ 49s 110ms/step - accuracy: 0.5381 - loss: 1.2008 - val

In [3]:
#Saving the  model
fer_json = model.to_json()
with open("fer.json", "w") as json_file:
    json_file.write(fer_json)
model.save("fer.h5")